In [599]:
'''import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


df = pd.read_csv('https://raw.githubusercontent.com/chefhaider/Fake-News-Detection-Kaggle/main/train.csv')


geolocator = Nominatim(user_agent = "geoapiExercises")
city_name = []




for val in df.location:
  if val == 'nan':
    city_name+=[' ']
    continue

  try:
    city_name+=[ geolocator.geocode(val)[-2].split(',')[-1] ]
  except TypeError:
    city_name+=[' ']
  except GeocoderTimedOut :
    city_name+=[' ']'''


'import pandas as pd\nfrom geopy.geocoders import Nominatim\nfrom geopy.exc import GeocoderTimedOut\n\n\ndf = pd.read_csv(\'https://raw.githubusercontent.com/chefhaider/Fake-News-Detection-Kaggle/main/train.csv\')\n\n\ngeolocator = Nominatim(user_agent = "geoapiExercises")\ncity_name = []\n\n\n\n\nfor val in df.location:\n  if val == \'nan\':\n    city_name+=[\' \']\n    continue\n\n  try:\n    city_name+=[ geolocator.geocode(val)[-2].split(\',\')[-1] ]\n  except TypeError:\n    city_name+=[\' \']\n  except GeocoderTimedOut :\n    city_name+=[\' \']'

In [600]:
#pip install pyspellchecker

In [601]:
 #nltk.download('stopwords')

In [602]:
import re
import string
import nltk
from nltk.corpus import stopwords

In [603]:
import pandas as pd
import requests

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [604]:
df = pd.read_csv('https://raw.githubusercontent.com/chefhaider/Fake-News-Detection-Kaggle/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/chefhaider/Fake-News-Detection-Kaggle/main/test.csv')

In [605]:
df.dropna(subset = ["keyword"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [606]:
'print shape of data set: ',df.shape,test.shape

('print shape of data set: ', (7552, 5), (3263, 4))

In [607]:
df.head()

,id,keyword,location,text,target
0,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
1,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
2,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
3,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
4,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [608]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [609]:
df.isnull().sum()

id             0
keyword        0
location    2472
text           0
target         0
dtype: int64

In [610]:
df['target'].value_counts()

0    4323
1    3229
Name: target, dtype: int64

In [611]:
#fig = plt.figure(figsize=(10,80))

#sns.countplot(y="keyword",order=df[df.target == 1]['keyword'].value_counts().index, hue="target",data = df)

In [612]:
# cleaning geo locations

In [613]:
from spellchecker import SpellChecker
import re
import string


spell = SpellChecker()


def clean_text(text):


    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'',text)


    html=re.compile(r'<.*?>')
    text = html.sub(r'',text)



    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)


    table=str.maketrans('','',string.punctuation)
    return text.translate(table)



df['text'] = df['text'].apply(lambda x: clean_text(x))
test['text'] = test['text'].apply(lambda x: clean_text(x))

In [614]:
'''corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)'''

'corrected_text = []\n    misspelled_words = spell.unknown(text.split())\n    for word in text.split():\n        if word in misspelled_words:\n            corrected_text.append(spell.correction(word))\n        else:\n            corrected_text.append(word)\n    return " ".join(corrected_text)'

In [615]:
from sklearn.feature_extraction.text import TfidfVectorizer


tf_idf_vect = TfidfVectorizer(ngram_range=(1,3)) # one,two and three gram vectorization
tf_idf_mat = tf_idf_vect.fit_transform(df['text'].values) # fit_transform vectorizer to dtrain['text']
tf_idf_mat_test = tf_idf_vect.transform(test['text'].values) # fit_transform vectorizer to dtest['text']
type(tf_idf_mat)
print(tf_idf_mat.get_shape())

(7552, 148500)


In [616]:
type(tf_idf_mat)

scipy.sparse.csr.csr_matrix

In [617]:
y.shape

(6041,)

In [618]:
def score(x):
  return df['keyword'].value_counts()[x]

df['keyword'] = df['keyword'].apply(lambda x: score(x))

In [619]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report





x, x_test, y, y_test = train_test_split(tf_idf_mat,df['target'],train_size=0.8, random_state = 0)

clf = MultinomialNB(alpha=1).fit(x, y)
pred = clf.predict(x_test)
 
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.73      0.96      0.83       858
           1       0.91      0.54      0.68       653

    accuracy                           0.78      1511
   macro avg       0.82      0.75      0.75      1511
weighted avg       0.81      0.78      0.76      1511



In [620]:
x_final, x_final_test, y_final, y_final_test = train_test_split(df['keyword'],df['target'],train_size=0.8, random_state = 0)

In [621]:
#x_final['proba'] = clf.predict_proba(x_test)

In [622]:
#x_final

In [626]:
clf.predict_proba(x_test)[0]

array([0.66031516, 0.33968484])

In [627]:
train = pd.DataFrame()
train['keyword'] = x_final_test
train['proba'] = [ x[0] for x in  clf.predict_proba(x_test) ]

In [ ]:
train = pd.DataFrame(data, columns = ['keyword', 'proba'])

In [628]:
from sklearn.tree import DecisionTreeClassifier


model = DecisionTreeClassifier()
model.fit(train,y_final_test)

pred = clf.predict(x_final_test)



ValueError: ignored